# Oja's Algorithm

## The following cell imports the .db file when run

In [1]:
!ls ../../K_cluster_analysis.db

../../K_cluster_analysis.db


In [2]:
import pandas as pd
import numpy as np
from sklearn import decomposition
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from matplotlib.pyplot import figure
import seaborn as sns
from sklearn.cluster import KMeans
import glob
%pylab inline

Populating the interactive namespace from numpy and matplotlib


From [the fast onvergence of incremental PCA](https://arxiv.org/pdf/1501.03796.pdf)

![](figures/OnlinePCAEquations.png)

### Explanation of Oja's update

* $V_n$ - the estimate of the top eigenvector at iteration $n$
* $\gamma_n$ - learning rate.
* $X_n$ The $n$th example
* $X_n X_n^T V_{n-1} = X_n (X_n \cdot V_{n-1})$

In [3]:
%%time
#run this to import
try:
    import dill
except:
    %pip install dill
    import dill
dill.load_session('../../K_cluster_analysis.db')
data=data[:,1:]  # remove prefix '1'
data.shape

CPU times: user 243 ms, sys: 1.33 s, total: 1.57 s
Wall time: 12.6 s


(2664671, 12)

### Estimate mean

## Calculating the regular mean onine

$\mu_n = \frac{\sum_{i=1}^n a_i}{n}$

$\frac{n}{n+1} \mu_n + \frac{a_{n+1}}{n+1}$

Equivalent to setting $\eta=1/(n+1)$

In [4]:
_mean=np.zeros(data.shape[1]) # initializing zero vector with 12 elements
_non_nan_incidence = np.zeros(data.shape[1])
eta=0.001 # setting learning rate

In [5]:
means=[] # initializing empty list for means
for i in range(data.shape[0]):  # looping through all data points
    vector=data[i,:] # selecting current vector
    _not_nan=~isnan(vector) # creating boolean vector based off current vector with 0's where NaNs are found and 1's otherwise
    _newmean=copy(_mean) # copying _mean vector
    eta=1/(i+1) # configuring learning rate with each iteration
    _newmean[_not_nan] = (1-eta)*_mean[_not_nan] + eta*vector[_not_nan]
    # above, we update only those elements of the mean corr. to non NaN elements in current vector.
    # if we have twelve elements in every vector, we must add all the non NaN values from the whole dataset
    # and divide each element by how many times the element was not NaN. 
    if i%100000 == 0:
        print(i,norm(_mean-_newmean))
    _mean=_newmean
    means.append(_mean)

0 1.0000604981699857
100000 5.736182578759251e-06
200000 6.937662597720766e-07
300000 7.521542888137032e-08
400000 5.9789299840198e-08
500000 2.842112320005564e-08
600000 1.9892053532317562e-08
700000 2.2275906975915026e-08
800000 1.3794310649967045e-07
900000 3.81451475294294e-08
1000000 9.455594422086783e-09
1100000 4.2651814163865296e-07
1200000 3.6439235675076137e-07
1300000 6.325536240915223e-08
1400000 1.228059522900131e-08
1500000 3.903372062654489e-09
1600000 1.3636588843548614e-08
1700000 5.2764013186954494e-08
1800000 5.024591349027317e-08
1900000 2.616893720235214e-07
2000000 7.084413616135357e-07
2100000 6.67039551507288e-07
2200000 2.592451029017278e-07
2300000 3.797223522909746e-08
2400000 4.364669281781259e-08
2500000 5.630394980643138e-07
2600000 1.5896442041951605e-07


In [6]:
k=100000
for i in range(0,len(means)-k,k):
    print(i,norm(means[i]-means[i+k]))

0 0.9112799400882033
100000 0.12404024365110486
200000 0.02652589667743528
300000 0.00316309581171782
400000 0.0009386316591947038
500000 0.0017968447548237705
600000 0.001803846358651171
700000 0.00272560816828956
800000 0.0013859855188258997
900000 0.007002621440201456
1000000 0.03328825955686318
1100000 0.03548923407269052
1200000 0.017958296653161808
1300000 0.0013654246877348774
1400000 0.0023432984021251333
1500000 0.0005841012329466819
1600000 0.004757069393662526
1700000 0.01580980415532421
1800000 0.007303714176165043
1900000 0.03310852384806099
2000000 0.020343131468545442
2100000 0.03430865144858992
2200000 0.01825453348303243
2300000 0.004033380789159255
2400000 0.012447000451012078
2500000 0.022686920604119448


In [7]:
norm(means[-1]-means[int(len(means)/2)])

0.1268358878205581

In [8]:
mean_last=means[-1]
norm(mean_last)

1.4727579784896112

In [9]:
eta*vector[_not_nan]

array([-9.00674042e-09, -4.50337021e-09,  3.75280851e-07])

In [10]:
_not_nan=~isnan(vector)

In [11]:
_not_nan

array([ True,  True,  True, False, False, False, False, False, False,
       False, False, False])

In [12]:
vector

array([-0.024, -0.012,  1.   ,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan])

In [36]:
np.nansum(vector)

0.964

In [13]:
vector[_not_nan]

array([-0.024, -0.012,  1.   ])

In [14]:
test_vec = data[2,:]
print(test_vec)
not_nn = ~isnan(test_vec)
print(not_nn)
short_vec = test_vec[not_nn]
print(short_vec)

[   nan    nan    nan    nan    nan    nan -0.011  0.     1.       nan
    nan    nan]
[False False False False False False  True  True  True False False False]
[-0.011  0.     1.   ]


### Implementing Oja's method

![](figures/OnlinePCAEquations.png)

In [ ]:
v= np.random.normal(0, 1, size = 12)
# print(v)
v = v / 10000
# print(v)

In [10]:
# Calculating the mean of the data points
print(data.shape)
data_mean = np.nanmean(data,axis = 0,keepdims = True)
print(data_mean)

# subtracting the means from the data matrix

print(data_mean.shape)

data = data - data_mean

(2664671, 12)
[[-0.02950949  0.00459708  0.95076457  0.1287518  -0.04651903  0.61195448
   0.05075128 -0.11779619  0.93551044  0.25047638 -0.0368738   0.8057973 ]]
(1, 12)


In [11]:
v= np.random.normal(0, 1, size = 12)
# print(v)
v = v / 10000
# print(v)

v_next_Nr = np.zeros((12))


for i in range(data.shape[0]):
    
    gamma_oja = max(0.001,1/(i+1)) # fixing learning rate [1]
    
    # Implementing Oja's rule equation
    # It involves an inner product. In the presence of NaNs, we look at it as
    # elementwise multiplication followed by nansum
    # Then we have a multiplication of gamma and the dot product with the current vector
    # We also add the previous weight vector to this.
    
    # we calculate the second term of the sum first
    oja_term_2 = gamma_oja * data[i,:] * np.nansum(np.multiply(data[i,:],v))
    
    # we find which the indices of elements that not nan
    _not_nan_term_2=~isnan(oja_term_2)
    
    # only updating those terms of the numerator which are not NaN
    v_next_Nr[_not_nan_term_2] = v[_not_nan_term_2] + oja_term_2[_not_nan_term_2]

#     if(i == 0):
#         print('At the 1st iteration, numerator is ',v_next_Nr)
#     if(i%100000==0):
#         print('the numerator is', v_next_Nr)
#     not_nn_oja = ~isnan(v_next_Nr)

    v_next_Dr = np.linalg.norm(v_next_Nr)
#     if(i%100000==0):
#         print('the denominator is', v_next_Dr)
    v_next = v_next_Nr/v_next_Dr

    v = v_next
    if(i%100000==0):
        print(i,v)

0 [ 0.          0.          0.          0.          0.          0.
 -0.29954059 -0.37849002  0.87579721  0.          0.          0.        ]
100000 [ 0.          0.          0.          0.          0.          0.
 -0.32638595 -0.30246893  0.89553602  0.          0.          0.        ]
200000 [ 0.          0.          0.          0.          0.          0.
 -0.3282908  -0.28355858  0.90101037  0.          0.          0.        ]
300000 [ 0.          0.          0.          0.          0.          0.
 -0.38383174  0.1258792   0.914736   -0.00704564  0.0010005   0.00591865]
400000 [ 0.0008284  -0.0008599   0.00220686  0.          0.          0.
 -0.30438313  0.76488248  0.56763873 -0.00704558  0.00100049  0.0059186 ]
500000 [ 0.00165292 -0.00254383  0.00571783  0.          0.          0.
 -0.33939048  0.81135118  0.47581068 -0.00704558  0.00100049  0.0059186 ]
600000 [ 0.00255498 -0.00351351  0.00788939  0.          0.          0.
 -0.27840146  0.85159633  0.4439702  -0.00704556  0.00100

### Obtaining the second second eigenvector

1. 'v' is the current eigenvector
2. project each point i onto v. call this projection i_proj_v
3. subtract, for each point, i_proj_v from i

In [12]:
v

array([ 0.16916344, -0.04079698,  0.11737491,  0.46364721,  0.03669775,
        0.11507357, -0.17431063,  0.26548009,  0.15865301, -0.62723891,
        0.06237903,  0.45058921])

In [13]:
print(i_1)

NameError: name 'i_1' is not defined

We use the equation to project vector a onto vector b:
$ \vec{a_1} = \vec{a}  \frac{\vec{b}}{||\vec{b}||} $

This is the magnitude of the projection vector.

In [14]:
# projecting i_1 onto v

data_perp_1 = np.zeros(data.shape)

v_mag = np.linalg.norm(v)

v_unit_vector = v/v_mag

for j in range(data.shape[0]):

    i_1 = data[j,:]

    proj_num = np.nansum(np.multiply(i_1,v))

    proj_not_nan = ~isnan(i_1)

    proj_den = np.linalg.norm(v[proj_not_nan])

    proj_mag = proj_num/proj_den

    proj_vector = proj_mag * v_unit_vector

    # print(proj_vector)

    # subtracting projection from datapoint

    data_perp_1[j,proj_not_nan] = i_1[proj_not_nan] - proj_vector[proj_not_nan]
    

### Oja's rule for second matrix

In [15]:
# calculating the mean of data_perp_1 and subtracting it from the data


print(data_perp_1.shape)
data_perp_1_mean = np.nanmean(data_perp_1,axis = 0,keepdims = True)
print(data_perp_1_mean)

# subtracting the means from the data matrix

print(data_perp_1_mean.shape)

data_perp_1 = data_perp_1 - data_perp_1_mean

(2664671, 12)
[[-0.0038939   0.00093909 -0.0027018   0.01117392  0.00088442  0.00277328
  -0.00178168  0.00271356  0.00162164 -0.00628616  0.00062516  0.00451578]]
(1, 12)


In [16]:
v_2= np.random.normal(0, 1, size = 12)
# print(v)
v_2 = v_2 / 10000
# print(v)

v_next_Nr_2 = np.zeros((12))


for i in range(data.shape[0]):
    
    gamma_oja_2 = max(0.001,1/(i+1)) # fixing learning rate [1]
    
    # Implementing Oja's rule equation
    # It involves an inner product. In the presence of NaNs, we look at it as
    # elementwise multiplication followed by nansum
    # Then we have a multiplication of gamma and the dot product with the current vector
    # We also add the previous weight vector to this.
    
    # we calculate the second term of the sum first
    oja_2_term_2 = gamma_oja_2 * data_perp_1[i,:] * np.nansum(np.multiply(data_perp_1[i,:],v_2))
    
    # we find which the indices of elements that not nan
    _not_nan_2_term_2=~isnan(oja_2_term_2)
    
    # only updating those terms of the numerator which are not NaN
    v_next_Nr_2[_not_nan_2_term_2] = v_2[_not_nan_2_term_2] + oja_2_term_2[_not_nan_2_term_2]

#     if(i == 0):
#         print('At the 1st iteration, numerator is ',v_next_Nr)
#     if(i%100000==0):
#         print('the numerator is', v_next_Nr)
#     not_nn_oja = ~isnan(v_next_Nr)

    v_next_Dr_2 = np.linalg.norm(v_next_Nr_2)
#     if(i%100000==0):
#         print('the denominator is', v_next_Dr)
    v_next_2 = v_next_Nr_2/v_next_Dr_2

    v_2 = v_next_2
    if(i%100000==0):
        print(i,v_2)

0 [ 0.37277669 -0.05731865 -0.39865916  0.44272782  0.03552602  0.30858963
  0.27503927 -0.18685365 -0.25201469 -0.18597132 -0.44459656  0.00129792]
100000 [ 0.7964066   0.38044611  0.4674886   0.02592613  0.00207626  0.01636902
  0.01419637 -0.01113982 -0.01355375 -0.01698885 -0.02644464 -0.00316093]
200000 [ 0.79616899  0.46464539  0.38341077  0.03821356  0.00304739  0.01883394
  0.01119919 -0.00624294 -0.01023646 -0.02322511 -0.02412523  0.00094293]
300000 [ 0.79619983  0.47632142  0.3687785   0.03927637  0.00313133  0.01902642
  0.00888749  0.00587675 -0.00430972 -0.02384473 -0.02384577  0.00155441]
400000 [ 0.79589296  0.47445424  0.36915287  0.03333238  0.0026608   0.01752333
  0.00292002  0.04954868  0.01683458 -0.02049555 -0.02410018 -0.0008048 ]
500000 [ 0.78969038  0.46611857  0.36914878  0.01996471  0.0016024   0.01406625
 -0.02059548  0.13139958  0.06132687 -0.01294899 -0.024457   -0.00599214]
600000 [ 0.73980385  0.42815484  0.34843755 -0.01071277 -0.00082741  0.00575964
 

In [19]:
# for a symmetric matrix eigenvectors must be normal
dot_prod_ev = np.dot(v,v_2)
print(dot_prod_ev)

0.21242311956092796


In [69]:
v2= np.random.normal(0, 1, size = 12)
print(v2)
v3= np.random.normal(0, 1, size = 12)
print(v3)

[-0.06512481  0.55507046 -0.06970197 -1.54765397  0.66028296  0.88860828
  0.48639555  0.38047019  0.30092569 -0.7525398   1.40319839 -0.08627907]
[ 0.17673735 -0.09013655 -0.1184975   0.14000183 -0.61194194  0.42845273
  0.48662751  1.14981189  0.8819502  -0.81194435 -0.55124646 -0.63579609]


In [83]:
print(_not_nan_term_2)

[False False False False False False  True  True  True False False False]


In [70]:
print(v2[_not_nan_term_2])

[-0.06512481  0.55507046 -0.06970197]


In [71]:
print(v3[_not_nan_term_2])

[ 0.17673735 -0.09013655 -0.1184975 ]


In [77]:
v4 = np.zeros((12))
print(v4)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [78]:
v4[_not_nan_term_2] = (v3[_not_nan_term_2]) + (v2[_not_nan_term_2])

In [80]:
print(v2,v3,v4)

[-0.06512481  0.55507046 -0.06970197 -1.54765397  0.66028296  0.88860828
  0.48639555  0.38047019  0.30092569 -0.7525398   1.40319839 -0.08627907] [ 0.17673735 -0.09013655 -0.1184975   0.14000183 -0.61194194  0.42845273
  0.48662751  1.14981189  0.8819502  -0.81194435 -0.55124646 -0.63579609] [0.         0.         0.         0.         0.         0.
 0.97302307 1.53028208 1.18287589 0.         0.         0.        ]


In [81]:
print(v2[_not_nan_term_2],v3[_not_nan_term_2],v4[_not_nan_term_2])

[0.48639555 0.38047019 0.30092569] [0.48662751 1.14981189 0.8819502 ] [0.97302307 1.53028208 1.18287589]


In [82]:
_not_nan_term_2

array([False, False, False, False, False, False,  True,  True,  True,
       False, False, False])

### References (links):

1. https://arxiv.org/pdf/1501.03796.pdf